In [5]:
import numpy as np
from collections import Counter
import xlsxwriter

import psycopg2
import pandas as pd
import pandas.io.sql as psql

In [7]:
class Aggregation:

    @staticmethod
    def get_preferences_matrices(estimates, direction):
        direction = list(map(lambda x: 1 if x == 'max' else 0, direction))
        estimates = np.array(estimates)
        preferences_matrices = []
        for col, k_v in enumerate(estimates.T):
            p_m = np.zeros((len(k_v), len(k_v)))
            for i in range(len(k_v)):
                for j in range(len(k_v)):
                    if direction[col] == 1:
                        p_m[i][j] = k_v[j] / (k_v[i] + k_v[j])
                    else:
                        p_m[i][j] = k_v[i] / (k_v[i] + k_v[j])

            preferences_matrices.append(p_m)

        return np.array(preferences_matrices)

    @staticmethod
    def get_total_preferences_matrix(preferences_matrices, coefficients):
        return sum(map(lambda x, y: x * y, preferences_matrices, coefficients))

    @staticmethod
    def Q_arithmetic(preferences_matrices, coefficients):
        return sum(map(lambda x, y: x * y, preferences_matrices, coefficients)) / len(preferences_matrices)

    @staticmethod
    def Q_median(preferences_matrices, coefficients):
        return np.median(np.array(list(map(lambda x, y: x * y, preferences_matrices, coefficients))), 0)

    @staticmethod
    def Q_minimax(preferences_matrices, coefficients):
        return (np.array(list(map(lambda x, y: x * y, preferences_matrices, coefficients))).min(0) + np.array(
            list(map(lambda x, y: x * y, preferences_matrices, coefficients))).max(0)) / 2

    @staticmethod
    def get_weights_matrix(matrix):
        n = len(matrix)
        temp = np.ndarray((n, n))
        for i in range(n):
            for j in range(n):
                if matrix[i][j] - matrix[j][i] >= 0:
                    temp[i][j] = matrix[i][j] - matrix[j][i]
                else:
                    temp[i][j] = 0

        return temp

    @staticmethod
    def get_adjacency_matrix(matrix):
        temp = Aggregation.zero_diag(matrix)
        return np.where(temp > temp.T, 1, 0)

    @staticmethod
    def zero_diag(matrix):
        temp = matrix.copy()
        np.fill_diagonal(temp, 0)
        return temp

    @staticmethod
    def transitive_closure(matrix, calc_lengths=False):
        n = len(matrix)
        Z = np.zeros((n, n))
        I = np.full((n, n), np.inf)
        T = np.where(matrix == Z, I, matrix)
        for k in range(n):
            for i in range(n):
                for j in range(n):
                    if T[i][k] < np.inf and T[k][j] < np.inf and T[i][k] + T[k][j] < T[i][j]:
                        if calc_lengths:
                            T[i][j] = T[i][k] + T[k][j]
                        else:
                            T[i][j] = 1

        T = np.where(T == I, Z, T)
        np.fill_diagonal(T, 0)

        return T

    @staticmethod
    def has_cycles(matrix):
        flag = False
        for i in range(1, len(matrix)):
            powered = np.linalg.matrix_power(matrix, i)
            if np.any(np.diagonal(powered)):
                flag = True

        return flag

    @staticmethod
    def demucron(matrix):
        if Aggregation.has_cycles(matrix):
            print('Граф содержит контуры, нельзя разбить на уровни с помощью адгоритма Демукрона\n')
            return []
        levels = []
        val = None
        alternatives = sorted(list(enumerate(np.sum(matrix, 1))), key=lambda x: x[1])
        for alt in alternatives:
            if val != alt[1]:
                val = alt[1]
                levels.append([])
            levels[-1].append(alt[0] + 1)

        print(f"Алгоритм Демукрона:\n{levels}\n")
        return levels

    @staticmethod
    def copeland(matrix):
        c = np.sum(matrix, 0) - np.sum(matrix, 1)
        levels = []
        val = None
        alternatives = sorted(list(enumerate(c)), key=lambda x: x[1], reverse=True)
        for alt in alternatives:
            if val != alt[1]:
                val = alt[1]
                levels.append([])
            levels[-1].append(alt[0] + 1)

        print(f"Процедура Коупленда:\n{c}\n{levels}\n")
        return levels

    @staticmethod
    def weight_difference(matrix):
        c = np.sum(matrix, 0) - np.sum(matrix, 1)
        levels = []
        val = None
        alternatives = sorted(list(enumerate(c)), key=lambda x: x[1], reverse=True)
        for alt in alternatives:
            if val != alt[1]:
                val = alt[1]
                levels.append([])
            levels[-1].append(alt[0] + 1)

        print(f"Процедура Разности Весов:\n{c}\n{levels}\n")
        return levels

    @staticmethod
    def weight_ratio(matrix):
        c = np.sum(matrix, 0) / np.sum(matrix, 1)
        levels = []
        val = None
        alternatives = sorted(list(enumerate(c)), key=lambda x: x[1], reverse=True)
        for alt in alternatives:
            if val != alt[1]:
                val = alt[1]
                levels.append([])
            levels[-1].append(alt[0] + 1)

        print(f"Процедура Отношения Весов:\n{c}\n{levels}")
        return levels

    @staticmethod
    def get_levels_with(matrix):
        weights_matrix = Aggregation.get_weights_matrix(matrix)
        adjacency_matrix = Aggregation.get_adjacency_matrix(matrix)
        print(adjacency_matrix)
        levels = [Aggregation.demucron(adjacency_matrix), Aggregation.copeland(adjacency_matrix),
                  Aggregation.weight_difference(weights_matrix), Aggregation.weight_ratio(weights_matrix)]

        return levels

In [176]:
class Solver:

    def __init__(self, df, direction, coefficients):
        self.df = df
        self.estimates = df.iloc[:, 1:]
        self.direction = direction
        self.coefficients = coefficients

    def with_selected_coeff(self, selected):
        estimates = np.array(self.estimates)[:, selected]
        direction = np.array(self.direction)[selected]
        coefficients = np.array(self.coefficients)[selected]
        preferences_matrices = Aggregation.get_preferences_matrices(estimates, direction)
        # Q_median = Aggregation.Q_median(preferences_matrices, coefficients)
        # median = Aggregation.get_levels_with(Q_median)
        Q_arithmetic = Aggregation.Q_arithmetic(preferences_matrices, coefficients)
        arith = Aggregation.get_levels_with(Q_arithmetic)

        return arith[1]

    def get_table(self, coeff_groups):
        tab = []
        for gr in coeff_groups:
            tab.append([gr, self.with_selected_coeff(gr)])

        with xlsxwriter.Workbook(f'Ранжирование дронов.xlsx') as workbook:
            worksheet = workbook.add_worksheet()

            v_align = {'valign': 'vcenter'}
            align_center = {'align': 'center'}
            text_wrap = {'text_wrap': True}
            align_center_format = workbook.add_format(v_align | align_center | text_wrap)

            length = 0
            headings = ['#']
            for i, col_data in enumerate(tab):
                if len(col_data[1]) > length:
                    length = len(col_data[1])
                heading = ', '.join(list(map(lambda x: f"K{x + 1}", col_data[0])))
                headings.append(heading)
                worksheet.write(1, i + 1, heading, align_center_format)
                for j, level in enumerate(col_data[1]):
                    worksheet.write(j + 2, i + 2, ', '.join(list(map(lambda x: f"A{x}", level))))

            for k in range(length):
                worksheet.write(k + 2, 1, k)

            worksheet.add_table(1, 1, length + 1, len(tab) + 1, {
                'columns': [{'header': h} for h in headings],
                'banded_rows': True,
                'banded_columns': True,
                'first_column': True,
                'style': 'Table Style Light 12'
            })

            worksheet.set_column(1, 100, 20)
            for i in range(1, 100):
                worksheet.set_row(i, 50, align_center_format)
            worksheet.set_row(1, 60, align_center_format)

            # worksheet.set_column(1, 1, 5)
            # worksheet.set_column(2, len(tab) + 1, 20)

            self.helicopters_table(1, 3 + len(tab), self.df, worksheet, workbook)

    def helicopters_table(self, row, col, df, worksheet, workbook):

        v_align = {'valign': 'vcenter'}
        align_center = {'align': 'center'}
        text_wrap = {'text_wrap': True}
        align_center_format = workbook.add_format(v_align | align_center | text_wrap)

        df_copy = df.copy()
        df_copy.insert(0, 'Num', [f'A{x + 1}' for x in range(len(df))])
        shape = np.array(df_copy).shape
        worksheet.add_table(row, col, row + shape[0], col + shape[1] - 1, {
            'data': df_copy.to_numpy(),
            'columns': [
                {'header': '#',
                 'format': align_center_format},
                {'header': 'Название',
                 'format': align_center_format},
                {'header': f'Вес\nКоэф. = {self.coefficients[0]}\n{self.direction[0]}',
                 'format': align_center_format},
                {'header': f'Время полета\nКоэф. = {self.coefficients[1]}\n{self.direction[1]}',
                 'format': align_center_format},
                {'header': f'Дальность\nКоэф. = {self.coefficients[2]}\n{self.direction[2]}',
                 'format': align_center_format},
                {'header': f'Высота\nКоэф. = {self.coefficients[3]}\n{self.direction[3]}',
                 'format': align_center_format},
                {'header': f'Скорость\nКоэф. = {self.coefficients[4]}\n{self.direction[4]}',
                 'format': align_center_format},
                {'header': f'Разрешение\nКоэф. = {self.coefficients[5]}\n{self.direction[5]}',
                 'format': align_center_format},
                {'header': f'FPS\nКоэф. = {self.coefficients[6]}\n{self.direction[6]}',
                 'format': align_center_format},
                {'header': f'Рейтинг\nКоэф. = {self.coefficients[7]}\n{self.direction[7]}',
                 'format': align_center_format},
                {'header': f'Стоимость\nКоэф. = {self.coefficients[8]}\n{self.direction[8]}',
                 'format': align_center_format},
            ]
        })

In [103]:

# estimates = [[10, 12, 4],
#              [15, 12, 3],
#              [15, 8, 2],
#              [5, 16, 1],
#              [20, 4, 2]],
# direction = ['min', 'min', 'max']

# estimates = [[3, 4, 2],
#              [2, 3, 4],
#              [3, 3, 2],
#              [2, 4, 4]],
# direction = ['max', 'max', 'max']
# coefficients = [1, 1, 1]

estimates = [[210, 27, 350, 30, 30, 1080, 30, 4, 10979],
             [570, 34, 6000, 5000, 19, 2160, 60, 4.8, 159599],
             [249, 31, 6000, 4000, 16, 2160, 30, 4.9, 86488],
             [245, 22, 500, 120, 10, 2160, 15, 4.6, 21990],
             [240, 15, 100, 100, 12, 1080, 30, 3.9, 3700],
             [249, 22, 500, 120, 9, 1080, 30, 5, 14990],
             [249, 30, 4000, 3000, 13, 1080, 60, 4.5, 49170],
             [249, 20, 300, 120, 5, 1080, 30, 3.9, 5190],
             [595, 31, 6000, 5000, 19, 2160, 60, 4.7, 132890],
             [80, 13, 20, 10, 8, 720, 30, 4.6, 22625]]
direction = ['min', 'max', 'max', 'max', 'max', 'max', 'max', 'max', 'min']
coefficients = [2, 2, 1, 1, 1, 2, 2, 1, 3]

In [4]:
preferences_matrices = Aggregation.get_preferences_matrices(estimates, direction)
total_preferences_matrix = Aggregation.get_total_preferences_matrix(preferences_matrices, coefficients)
Q_arithmetic = Aggregation.Q_arithmetic(preferences_matrices, coefficients)
Q_median = Aggregation.Q_median(preferences_matrices, coefficients)
Q_minimax = Aggregation.Q_minimax(preferences_matrices, coefficients)

In [441]:
print('P_sum\n')
total = Aggregation.get_levels_with(total_preferences_matrix)
print('_________________________________________________________________________\n')
print('Q_arithmetic\n')
arithmetic = Aggregation.get_levels_with(Q_arithmetic)
print('_________________________________________________________________________\n')
print('Q_median\n')
median = Aggregation.get_levels_with(Q_median)
print('_________________________________________________________________________\n')
print('Q_minimax\n')
minimax = Aggregation.get_levels_with(Q_minimax)
print('_________________________________________________________________________\n')

P_sum

Граф содержит контуры, нельзя разбить на уровни с помощью адгоритма Демукрона

Процедура Коупленда:
[ 3  1  5 -7  1 -5  9 -1  3 -9]
[[7], [3], [1, 9], [2, 5], [8], [6], [4], [10]]

Процедура Разности Весов:
[  4.28041115   2.0251474    6.2835196   -6.02828225   2.72633617
  -2.38914097   8.52389928   1.82579663   2.1793191  -19.4270061 ]
[[7], [3], [1], [5], [9], [2], [8], [6], [4], [10]]

Процедура Отношения Весов:
[ 5.7198433   2.01410103 20.54759538  0.22551064  1.99631778  0.5488305
         inf  1.53051942  2.24808526  0.        ]
[[7], [3], [1], [9], [2], [5], [8], [6], [4], [10]]
_________________________________________________________________________

Q_arithmetic

Граф содержит контуры, нельзя разбить на уровни с помощью адгоритма Демукрона

Процедура Коупленда:
[ 3  1  5 -7  1 -5  9 -1  3 -9]
[[7], [3], [1, 9], [2, 5], [8], [6], [4], [10]]

Процедура Разности Весов:
[ 0.47560124  0.22501638  0.69816884 -0.66980914  0.30292624 -0.26546011
  0.94709992  0.20286629  0.24

/var/folders/23/hzkz91rj3pq502cfq0_65r0c0000gn/T/ipykernel_25373/3516334113.py:141: RuntimeWarning: divide by zero encountered in true_divide
  c = np.sum(matrix, 0) / np.sum(matrix, 1)


# Чтение из базы

In [104]:
DB_CFG = {
    'dbname': 'diploma',
    'user': 'diploma',
    'password': 'diploma',
    'host': 'localhost'
}

In [177]:
conn = psycopg2.connect(**DB_CFG)
df = psql.read_sql('''SELECT name,
                               weight,
                               duration,
                               distance,
                               height,
                               speed,
                               pixels,
                               fps,
                               rating,
                               price
                        FROM helicopters
                        ORDER BY id''', conn)


In [178]:
solver = Solver(df, direction, coefficients)

In [179]:
solver.get_table([[0, 1], [0, 1, 2], [0, 1, 5, 6, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8]])

[[0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0]
 [1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0]
 [0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0]
 [1 1 0 1 1 1 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 0]
 [1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0]
 [0 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0]
 [1 1 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 0]
 [1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0]
 [0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 0]
 [1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 1 1

/var/folders/23/hzkz91rj3pq502cfq0_65r0c0000gn/T/ipykernel_98525/679168550.py:141: RuntimeWarning: divide by zero encountered in true_divide
  c = np.sum(matrix, 0) / np.sum(matrix, 1)


In [113]:
df.insert(0, 'Num', [f'A{x + 1}' for x in range(len(df))])

In [114]:
df

,Num,name,weight,duration,distance,height,speed,pixels,fps,rating,price
0,A1,"Квадрокоптер HIPER Paladin FPV, черный/серый",209,16,60,60,13,480,30,4.8,6690
1,A2,"Квадрокоптер DJI Mini 2, серый",249,31,6000,4000,16,1080,60,4.8,78890
2,A3,"Квадрокоптер Syma X8Pro, белый",600,9,70,70,7,720,30,4.4,10057
3,A4,"Квадрокоптер Xiaomi MiTu Minidrone 720P, белый...",88,10,50,25,23,720,30,4.2,6000
4,A5,"Квадрокоптер Xiaomi MiTu Minidrone 720P, белый...",88,10,50,25,13,720,30,4.2,5998
5,A6,"Квадрокоптер Syma X30, серый",210,27,350,350,30,1080,30,4.0,13964
6,A7,"Квадрокоптер Syma W1 Pro, белый",300,17,50,70,16,1080,30,3.6,18037
7,A8,"Квадрокоптер Syma X23W, белый",132,7,70,70,13,720,30,4.3,4200
8,A9,"Квадрокоптер Eachine E58 WIFI FPV 2MP, black",80,9,100,100,17,720,30,4.0,4190
9,A10,"Квадрокоптер HIPER Falcon X FPV, черный/фиолет...",101,6,60,60,9,480,30,4.0,4627
